In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, utils, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the base directory and categories
base_dir = Path("/content/drive/MyDrive/DATASET_BC")
categories = ['normal', 'benign', 'malignant']

# Set the input image size and batch size
input_shape = (224, 224, 3)
batch_size = 32
num_classes = len(categories)

# Create ImageDataGenerators for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Splitting 20% of the data for validation
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # Ensure images are resized to match the input size
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Print class names to ensure proper categorization
print("Class indices:", train_generator.class_indices)


Found 1271 images belonging to 3 classes.
Found 317 images belonging to 3 classes.
Class indices: {'benign': 0, 'malignant': 1, 'normal': 2}


In [ ]:
def build_alexnet(input_shape=(224, 224, 3), num_classes=3):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Changed to match your dataset
    ])
    return model
def build_googlenet(input_shape=(224, 224, 3), num_classes=3):
    input_img = layers.Input(shape=input_shape)

    # Convolutional Layer
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(input_img)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

    # Add more layers as per requirement

    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(num_classes, activation='softmax')(x)  # Changed to match your dataset

    model = models.Model(inputs=input_img, outputs=output)
    return model


In [ ]:
def load_pretrained_models(input_shape=(224, 224, 3), num_classes=3):
    models_dict = {
        "AlexNet": build_alexnet(input_shape, num_classes),
        "GoogLeNet": build_googlenet(input_shape, num_classes),
        "ShuffleNet": applications.MobileNet(input_shape=input_shape, weights=None, classes=num_classes),
        "ResNet-50": applications.ResNet50(input_shape=input_shape, weights=None, classes=num_classes),
        "SqueezeNet": applications.MobileNetV2(input_shape=input_shape, weights=None, classes=num_classes),
        # "ResNet-18": applications.ResNet18(input_shape=input_shape, weights=None, classes=num_classes),
        "ResNet-101": applications.ResNet101(input_shape=input_shape, weights=None, classes=num_classes),
        "XceptionNet": applications.Xception(input_shape=input_shape, weights=None, classes=num_classes)
    }
    return models_dict

# Train and evaluate the models
models_dict = load_pretrained_models()

for name, model in models_dict.items():
    print(f"Training {name}...")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_generator, epochs=10, validation_data=validation_generator, verbose=1)
    test_loss, test_accuracy = model.evaluate(validation_generator)
    print(f"{name} Test accuracy: {test_accuracy}\n")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training AlexNet...
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 604s 13s/step - accuracy: 0.4476 - loss: 2.6264 - val_accuracy: 0.6845 - val_loss: 0.7591
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 313ms/step - accuracy: 0.6294 - loss: 0.8328 - val_accuracy: 0.6625 - val_loss: 0.7274
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 317ms/step - accuracy: 0.6841 - loss: 0.7209 - val_accuracy: 0.6972 - val_loss: 0.7281
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 314ms/step - accuracy: 0.6721 - loss: 0.7181 - val_accuracy: 0.6845 - val_loss: 0.6741
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 306ms/step - accuracy: 0.6881 - loss: 0.6256 - val_accuracy: 0.7098 - val_loss: 0.6363
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 307ms/step - accuracy: 0.7187 - loss: 0.6054 - val_accuracy: 0.6909 - val_loss: 0.6358
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 307ms/step - accuracy: 0.7406 - loss: 0.5797 - val_accuracy: 0.6845 - val_loss: 0.8597
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 321ms/step - accuracy: 0.7946 - loss: 0.4840 - val_accuracy: 0.7350